In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import re
import unicodedata
import string
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline
torch.manual_seed(1)

In [ ]:
class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:'SOS',1:'EOS'}
        self.n_words = 2
        
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
## Preparing data for training
# say we want eng-fra, then reverse implies fra-eng
def readLangFiles(lang1,lang2,file_base_dir=None,reverse=False,read_limit=20000):
    if not file_base_dir:
        raise ValueError('Enter valid base directory')
    
    fname = file_base_dir + '{}-{}.txt'.format(lang1,lang2)
    with open(fname,'r',encoding='utf-8') as f:
        lines = [[normalizeString(x) for x in p.strip().split('\t')] for p in f.readlines()]
    
    random.shuffle(lines)
    lines = lines[:read_limit]
    if reverse:
        lines = [[y,x] for x,y in lines]
        inp_lang = Lang(lang2)
        out_lang = Lang(lang1)
    else:
        inp_lang = Lang(lang1)
        out_lang = Lang(lnag2)
        
    return inp_lang,out_lang,lines

In [ ]:
# Remember to add the full path
base_path = 'seq2seq_data/'

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return (len(p[0].split(' ')) < MAX_LENGTH) and \
        (len(p[1].split(' ')) < MAX_LENGTH) and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1,lang2,reverse=False):
    inp_lang,out_lang,pairs = readLangFiles(lang1,lang2,base_path,reverse,read_limit=-1)
    print("Read %s sentence pairs"%len(pairs))
    filteredPairs = filterPairs(pairs)
    print("Pairs remaining after filtering: %s"%len(filteredPairs))
    for p in filteredPairs:
        inp_lang.addSentence(p[0])
        out_lang.addSentence(p[1])
    print("Counted words:")
    print(inp_lang.name, inp_lang.n_words)
    print(out_lang.name, out_lang.n_words)
    return inp_lang, out_lang, filteredPairs

inp_lang, out_lang, pairs = prepareData('eng', 'fra', True)
from random import shuffle
shuffle(pairs)
print(pairs[0],type(pairs[0]))
#print(random.choice(pairs))

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self,inp_dim,hidden_dim):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_dim
        
        self.embedding = nn.Embedding(inp_dim,hidden_dim)
        self.gru = nn.GRU(hidden_dim,hidden_dim)
        
    def forward(self,inp, hidden):
        embedded = self.embedding(inp).view(1,1,-1)
        output,hidden = self.gru(embedded, hidden)
        return output,hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size) # why 3-d tensor?

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self,output_size,hidden_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, inp, hidden):
        output = self.embedding(inp).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [ ]:
# Prepared Training Data
# Things required - input tensor with indices of each word in the sentence   
def index2Tensor(lang,word):
    return torch.tensor(lang.word2index[word])
    
def sentence2Tensor(lang,sentence):
    words = sentence.split()
    sentenceTensor = torch.zeros((len(words)+1,1),dtype=torch.long)
    for i,word in enumerate(words):
        sentenceTensor[i] = index2Tensor(lang,word)
    sentenceTensor[-1] = torch.tensor(1) # for 'EOS'
    return sentenceTensor
    
def prepareTensors(lang1,lang2,pairs):
    input_tensor = sentence2Tensor(lang1,pairs[0])
    target_tensor = sentence2Tensor(lang2,pairs[1])
    return input_tensor,target_tensor

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self,hidden_dim,output_dim,dropout_p=0.1,max_length=MAX_LENGTH):
        super(AttnDecoderRNN,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(self.output_dim,self.hidden_dim)
        self.attn = nn.Linear(self.hidden_dim*2,max_length)
        self.attn_combine = nn.Linear(self.hidden_dim*2,self.hidden_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(self.hidden_dim,self.hidden_dim)
        self.out = nn.Linear(self.hidden_dim,self.output_dim)
    
    def forward(self,inp,hidden,encoder_outputs):
        embedded = self.embedding(inp).view(1,1,-1)
        embedded = self.dropout(embedded)
        attn_weights = F.softmax(self.attn(torch.cat(
                  (embedded[0],hidden[0]),1)),dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                    encoder_outputs.unsqueeze(0))
        out = torch.cat((embedded[0],attn_applied[0]),1)
        out = self.attn_combine(out).unsqueeze(0)

        out = F.relu(out)
        out, hidden = self.gru(out, hidden)

        out = F.log_softmax(self.out(out[0]), dim=1)
        return out, hidden, attn_weights    

    def initHidden(self):
        return torch.zeros(1,1,self.hidden_dim)

In [ ]:
EOS_token = 1
teacher_forcing_ratio = 0.5

def train(input_tensor,target_tensor,encoder,decoder,encoder_optimizer, \
          decoder_optimizer,criterion,max_length=MAX_LENGTH):
  
    target_length = target_tensor.size(0)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)#, device=device)
    encoder_hidden = encoder.initHidden()
    
    for i,inp in enumerate(input_tensor):
        # just a simple hack
        if (input_tensor.shape[0] <= MAX_LENGTH):
            encoder_out,encoder_hidden = encoder(inp,encoder_hidden)
            encoder_outputs[i] = encoder_out[0,0]

    decoder_hidden = encoder_hidden
    decoder_input = torch.tensor([0])
    
    loss = 0
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item()/target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from random import randint

%matplotlib inline

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    #training_pairs = [prepareTensors(inp_lang,out_lang,random.choice(pairs))
    #                  for i in range(n_iters)]
    
    criterion = nn.NLLLoss()
    num_pairs = len(pairs)

    for i in range(1, n_iters + 1):
        index = randint(0,num_pairs-1)
        input_tensor,target_tensor = prepareTensors(inp_lang, out_lang, pairs[index])
        
        loss = train(input_tensor, target_tensor, encoder, \
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, i / n_iters),
                                         i, i / n_iters * 100, print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    
    showPlot(plot_losses)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(inp_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, out_lang.n_words, dropout_p=0.1)

In [ ]:
## save the model 
trainIters(encoder1, attn_decoder1, 75000, print_every=5000)
torch.save(encoder1.state_dict(),os.getcwd()+'encoder.pt')
torch.save(attn_decoder1.state_dict(),os.getcwd()+'attn_decoder.pt')

In [ ]:
## reload model
encoder_saved = EncoderRNN(inp_lang.n_words, hidden_size)
encoder_saved.load_state_dict(torch.load(os.getcwd()+'encoder.pt'))
attn_decoder_saved = AttnDecoderRNN(hidden_size, out_lang.n_words, dropout_p=0.1)
attn_decoder_saved.load_state_dict(torch.load(os.getcwd()+'attn_decoder.pt'))

In [ ]:
attn_decoder_saved

In [ ]:
EOS_token = 1
def evaluate(encoder,decoder,inp_sentence,max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = sentence2Tensor(inp_lang,inp_sentence)
        input_length = input_tensor.shape[0]

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size)#, device=device)
        encoder_hidden = encoder.initHidden()

        for i,inp in enumerate(input_tensor):
            # just a simple hack
            encoder_out,encoder_hidden = encoder(inp,encoder_hidden)
            encoder_outputs[i] += encoder_out[0,0]

        decoder_hidden = encoder_hidden
        decoder_input = torch.tensor([[0]])
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output,decoder_hidden,decoder_attention = decoder( \
                decoder_input,decoder_hidden,encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]    

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = pairs[i]
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        
evaluateRandomly(encoder1, attn_decoder1, 100)

Function to check encoder-decoder without Attention Model

In [ ]:
def checkEncoderDecoder(inp_pairs,hidden_size=256,learning_rate=1e-4):
    
    # Encoder operates on the French Language
    encoderRnn = EncoderRNN(inp_lang.n_words,hidden_size)
    encoder_optimizer = optim.SGD(encoderRnn.parameters(), lr=learning_rate)
    
    # Decoder operates on the English Language
    decoderRnn = DecoderRNN(out_lang.n_words,hidden_size)
    decoder_optimizer = optim.SGD(decoderRnn.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()
    total_loss = []
    for pair in inp_pairs:
        encoder_hidden = encoderRnn.initHidden()
        inp_tensor,target_tensor = prepareTensors(inp_lang, out_lang, pair)
        for inp in inp_tensor:
            encoder_out,encoder_hidden = encoderRnn(inp,encoder_hidden)

        decoder_hidden = encoder_hidden
        decoder_inp = torch.tensor([0])

        loss = 0
        # by default enforce teacher learning
        for t in target_tensor:
            decoder_out,decoder_hidden = decoderRnn(decoder_inp,decoder_hidden)
            loss += criterion(decoder_out,t)
            decoder_inp = t

        total_loss.append(loss)
        loss.backward()
        
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
    
    plt.plot(total_loss)
    plt.show()

In [ ]:
checkEncoderDecoder(pairs[:10000])